# Stanford Bunny - 3D Point Cloud Analysis & Volumetric Measurements

**Pipeline:**
1. Point Cloud Loading & 3D Visualization
2. Mesh Reconstruction (Convex Hull, Marching Cubes)
3. 3D Gaussian Splatting (3DGS)
4. Volume Measurement (Divergence Theorem, Signed Tetrahedra, Monte Carlo, Voxelization)
5. Comparison & Analysis

# --- Environment Setup (Colab / Local) ---
import os

try:
    from google.colab import drive
    drive.mount('/content/drive')
    IN_COLAB = True
    PROJECT_DIR = '/content/drive/MyDrive/Volumetric_measurements'
    WORK_DIR = '/content/volumetric_work'
    os.makedirs(WORK_DIR, exist_ok=True)
    os.chdir(WORK_DIR)
    # Extract bunny data from Drive
    import tarfile
    src = os.path.join(PROJECT_DIR, 'bunny.tar.gz')
    if not os.path.exists('bunny'):
        print(f'Extracting {src} ...')
        with tarfile.open(src, 'r:gz') as tar:
            tar.extractall('.')
        print('Done!')
    else:
        print('bunny/ already exists.')
except ImportError:
    IN_COLAB = False
    WORK_DIR = os.path.dirname(os.path.abspath('volumetric.ipynb'))
    if not WORK_DIR:
        WORK_DIR = os.getcwd()
    os.chdir(WORK_DIR)
    PROJECT_DIR = WORK_DIR
    print('Running locally (not Colab)')
    if not os.path.exists('bunny') and os.path.exists('bunny.tar.gz'):
        import tarfile
        with tarfile.open('bunny.tar.gz', 'r:gz') as tar:
            tar.extractall('.')
        print('Extracted bunny.tar.gz')

print(f'Working directory: {os.getcwd()}')

In [ ]:
# Install packages (Colab only)
if IN_COLAB:
    !pip install -q plotly trimesh rtree scikit-image scikit-learn tqdm shapely
else:
    print('Local environment - skipping pip install')

In [ ]:
import sys, warnings
import numpy as np
import scipy
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from scipy.spatial import ConvexHull, KDTree
from scipy.ndimage import gaussian_filter
from sklearn.neighbors import NearestNeighbors
import trimesh
from tqdm import tqdm
from pathlib import Path
import json

warnings.filterwarnings('ignore')
np.random.seed(42)

print(f'Python   : {sys.version}')
print(f'NumPy    : {np.__version__}')
print(f'SciPy    : {scipy.__version__}')
print(f'Trimesh  : {trimesh.__version__}')
print('\nAll libraries loaded!')

---
## 1. Data Loading & Exploration

Stanford Bunny dataset:
- `bunny/data/` : 10 range scan PLY files (raw point clouds)
- `bunny/reconstruction/` : 4 reconstructed meshes at different resolutions

In [ ]:
def parse_ply(filepath):
    """Parse ASCII PLY file -> vertices (N,D), faces (M,3), property names."""
    with open(filepath, 'r') as f:
        lines = f.readlines()

    n_verts = n_faces = 0
    header_end = 0
    props = []
    in_vertex_elem = False

    for i, line in enumerate(lines):
        s = line.strip()
        if s.startswith('element vertex'):
            n_verts = int(s.split()[-1])
            in_vertex_elem = True
        elif s.startswith('element'):
            if 'face' in s:
                n_faces = int(s.split()[-1])
            in_vertex_elem = False
        elif s.startswith('property') and in_vertex_elem:
            props.append(s.split()[-1])
        elif s == 'end_header':
            header_end = i + 1
            break

    verts = np.array(
        [[float(x) for x in lines[header_end + j].split()] for j in range(n_verts)]
    )

    faces = None
    if n_faces > 0:
        face_start = header_end + n_verts
        faces = np.array(
            [[int(x) for x in lines[face_start + j].split()[1:4]] for j in range(n_faces)]
        )

    return verts, faces, props


BASE = Path('bunny/reconstruction')

mesh_files = {
    'Full (35k)'  : 'bun_zipper.ply',
    'Res2 (8k)'   : 'bun_zipper_res2.ply',
    'Res3 (1.9k)' : 'bun_zipper_res3.ply',
    'Res4 (453)'  : 'bun_zipper_res4.ply',
}

meshes = {}
for label, fname in mesh_files.items():
    v, f, p = parse_ply(BASE / fname)
    meshes[label] = {'vertices': v[:, :3], 'faces': f, 'props': p}
    print(f'{label:16s} | {v.shape[0]:>6,} verts | {f.shape[0]:>6,} faces')

# Full mesh for visualization, Res2 for heavy computation
WORK = 'Full (35k)'
vertices = meshes[WORK]['vertices']
faces    = meshes[WORK]['faces']

# Lighter mesh for Monte Carlo / Voxelization (much faster)
WORK_LIGHT = 'Res2 (8k)'
vertices_light = meshes[WORK_LIGHT]['vertices']
faces_light    = meshes[WORK_LIGHT]['faces']

print(f'\nVisualization mesh: {WORK} ({len(vertices):,} verts)')
print(f'Computation mesh : {WORK_LIGHT} ({len(vertices_light):,} verts)')
print(f'  X: [{vertices[:,0].min():.4f}, {vertices[:,0].max():.4f}]')
print(f'  Y: [{vertices[:,1].min():.4f}, {vertices[:,1].max():.4f}]')
print(f'  Z: [{vertices[:,2].min():.4f}, {vertices[:,2].max():.4f}]')

In [ ]:
centroid = vertices.mean(axis=0)
dists_from_center = np.linalg.norm(vertices - centroid, axis=1)
nbrs = NearestNeighbors(n_neighbors=2).fit(vertices)
nn_dists = nbrs.kneighbors(vertices)[0][:, 1]
bbox_dims = vertices.max(axis=0) - vertices.min(axis=0)

print('=== Point Cloud Statistics ===')
print(f'  Points           : {len(vertices):,}')
print(f'  Centroid         : ({centroid[0]:.5f}, {centroid[1]:.5f}, {centroid[2]:.5f})')
print(f'  BBox dimensions  : {bbox_dims[0]:.4f} x {bbox_dims[1]:.4f} x {bbox_dims[2]:.4f}')
print(f'  BBox volume      : {np.prod(bbox_dims):.6f}')
print(f'  Mean NN distance : {nn_dists.mean():.6f}')

---
## 2. Interactive 3D Visualization

In [ ]:
fig_pc = go.Figure(data=[go.Scatter3d(
    x=vertices[:, 0], y=vertices[:, 2], z=vertices[:, 1],
    mode='markers',
    marker=dict(size=1.2, color=vertices[:, 1], colorscale='Turbo',
                showscale=True, colorbar=dict(title='Height (Y)')),
)])
fig_pc.update_layout(
    title='Stanford Bunny - Point Cloud',
    scene=dict(aspectmode='data',
               xaxis_title='X', yaxis_title='Z', zaxis_title='Y'),
    width=900, height=700,
)
fig_pc.show()

In [ ]:
fig_mesh = go.Figure(data=[go.Mesh3d(
    x=vertices[:, 0], y=vertices[:, 2], z=vertices[:, 1],
    i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
    intensity=vertices[:, 1], colorscale='Turbo',
    showscale=True, colorbar=dict(title='Height'),
    flatshading=True,
    lighting=dict(ambient=0.4, diffuse=0.7, specular=0.3, roughness=0.5),
    lightposition=dict(x=100, y=200, z=100),
)])
fig_mesh.update_layout(
    title='Stanford Bunny - Triangle Mesh (35k vertices, 69k faces)',
    scene=dict(aspectmode='data',
               xaxis_title='X', yaxis_title='Z', zaxis_title='Y'),
    width=900, height=700,
)
fig_mesh.show()

---
## 3. Mesh Reconstruction from Point Cloud

Three reconstruction methods:
1. **Convex Hull** - tightest convex envelope (upper bound)
2. **Marching Cubes** - implicit surface from voxelized distance field
3. **Comparison** with original pre-built mesh

In [ ]:
hull = ConvexHull(vertices)

fig_hull = go.Figure(data=[go.Mesh3d(
    x=vertices[:, 0], y=vertices[:, 2], z=vertices[:, 1],
    i=hull.simplices[:, 0], j=hull.simplices[:, 1], k=hull.simplices[:, 2],
    opacity=0.4, color='cyan', flatshading=True,
)])
fig_hull.update_layout(
    title=f'Convex Hull ({hull.simplices.shape[0]} faces, Volume={hull.volume:.6f})',
    scene=dict(aspectmode='data'),
    width=800, height=600,
)
fig_hull.show()
print(f'Convex Hull: {hull.simplices.shape[0]} faces, Volume = {hull.volume:.8f}')

In [ ]:
from skimage.measure import marching_cubes

print('Building signed distance field...')
resolution = 80
pad = 0.005
mins = vertices.min(axis=0) - pad
maxs = vertices.max(axis=0) + pad

grid = np.zeros((resolution, resolution, resolution), dtype=np.float32)
indices = ((vertices - mins) / (maxs - mins) * (resolution - 1)).astype(int)
indices = np.clip(indices, 0, resolution - 1)
grid[indices[:, 0], indices[:, 1], indices[:, 2]] = 1.0

grid_smooth = gaussian_filter(grid, sigma=1.5)
level = grid_smooth.max() * 0.15
mc_verts, mc_faces, mc_normals, _ = marching_cubes(grid_smooth, level=level)
mc_verts = mc_verts / (resolution - 1) * (maxs - mins) + mins

print(f'Marching Cubes: {len(mc_verts):,} verts, {len(mc_faces):,} faces')

fig_mc = go.Figure(data=[go.Mesh3d(
    x=mc_verts[:, 0], y=mc_verts[:, 2], z=mc_verts[:, 1],
    i=mc_faces[:, 0], j=mc_faces[:, 1], k=mc_faces[:, 2],
    intensity=mc_verts[:, 1], colorscale='Plasma',
    flatshading=True, showscale=False,
)])
fig_mc.update_layout(
    title='Mesh Reconstruction (Marching Cubes from Distance Field)',
    scene=dict(aspectmode='data'), width=800, height=600,
)
fig_mc.show()

In [ ]:
fig_comp = make_subplots(
    rows=1, cols=3,
    specs=[[{'type': 'scene'}, {'type': 'scene'}, {'type': 'scene'}]],
    subplot_titles=['Original Mesh', 'Convex Hull', 'Marching Cubes'],
)
fig_comp.add_trace(go.Mesh3d(
    x=vertices[:,0], y=vertices[:,2], z=vertices[:,1],
    i=faces[:,0], j=faces[:,1], k=faces[:,2],
    intensity=vertices[:,1], colorscale='Turbo', showscale=False, flatshading=True,
), row=1, col=1)
fig_comp.add_trace(go.Mesh3d(
    x=vertices[:,0], y=vertices[:,2], z=vertices[:,1],
    i=hull.simplices[:,0], j=hull.simplices[:,1], k=hull.simplices[:,2],
    color='cyan', flatshading=True, showscale=False,
), row=1, col=2)
fig_comp.add_trace(go.Mesh3d(
    x=mc_verts[:,0], y=mc_verts[:,2], z=mc_verts[:,1],
    i=mc_faces[:,0], j=mc_faces[:,1], k=mc_faces[:,2],
    intensity=mc_verts[:,1], colorscale='Plasma', showscale=False, flatshading=True,
), row=1, col=3)
for sc in ['scene', 'scene2', 'scene3']:
    fig_comp.update_layout(**{sc: dict(aspectmode='data')})
fig_comp.update_layout(title='Mesh Reconstruction Comparison', width=1400, height=500)
fig_comp.show()

---
## 4. 3D Gaussian Splatting (3DGS)

3DGS represents a scene as a collection of 3D Gaussian distributions.

Each Gaussian:
- **Position** $\mu \in \mathbb{R}^3$
- **Covariance** $\Sigma \in \mathbb{R}^{3 \times 3}$ (shape/orientation)
- **Opacity** $\alpha \in [0, 1]$
- **Color** $c \in \mathbb{R}^3$

$$G(\mathbf{x}) = \alpha \cdot \exp\left(-\tfrac{1}{2}(\mathbf{x}-\mu)^T \Sigma^{-1} (\mathbf{x}-\mu)\right)$$

Rendering: each 3D Gaussian is **splatted** onto the 2D image plane and alpha-composited front-to-back.

In [ ]:
class GaussianModel:
    """Simplified 3D Gaussian Splatting model."""

    def __init__(self, points, k_neighbors=8, sample_rate=1.0):
        n = int(len(points) * sample_rate)
        idx = np.random.choice(len(points), n, replace=False)
        self.means = points[idx].copy()
        self.n = n

        y_norm = (self.means[:, 1] - self.means[:, 1].min()) / \
                 (self.means[:, 1].max() - self.means[:, 1].min() + 1e-10)
        cmap = cm.get_cmap('turbo')
        self.colors = cmap(y_norm)[:, :3]
        self.opacities = np.ones(n) * 0.85

        print(f'  Estimating covariances for {n:,} Gaussians (k={k_neighbors})...')
        nbrs = NearestNeighbors(n_neighbors=k_neighbors + 1).fit(self.means)
        _, knn_indices = nbrs.kneighbors(self.means)

        self.covariances = np.zeros((n, 3, 3))
        self.scales = np.zeros((n, 3))
        self.rotations = np.zeros((n, 3, 3))

        for i in range(n):
            neighbors = self.means[knn_indices[i, 1:]]
            centered = neighbors - self.means[i]
            cov = (centered.T @ centered) / k_neighbors + np.eye(3) * 1e-8
            self.covariances[i] = cov
            eigvals, eigvecs = np.linalg.eigh(cov)
            self.scales[i] = np.sqrt(np.maximum(eigvals, 1e-10))
            self.rotations[i] = eigvecs

        # Pre-compute inverse covariances for rendering speed
        self.cov_invs = np.linalg.inv(self.covariances)
        print(f'  Done. Mean scale: {self.scales.mean(axis=0)}')

# Use lighter mesh (Res2) for Gaussian model - much faster
print('Building 3D Gaussian model from Res2 mesh (50% sample)...')
gm = GaussianModel(vertices_light, k_neighbors=8, sample_rate=0.5)
print(f'\nGaussian Model: {gm.n:,} Gaussians')
print(f'  Scale range: [{gm.scales.min():.6f}, {gm.scales.max():.6f}]')

In [ ]:
def make_ellipsoid(mean, scales, rotation, n_pts=20):
    u = np.linspace(0, 2*np.pi, n_pts)
    v = np.linspace(0, np.pi, n_pts)
    x = np.outer(np.cos(u), np.sin(v))
    y = np.outer(np.sin(u), np.sin(v))
    z = np.outer(np.ones_like(u), np.cos(v))
    sphere = np.stack([x.ravel(), y.ravel(), z.ravel()], axis=1)
    ellipsoid = sphere * scales[None, :] @ rotation.T + mean[None, :]
    return ellipsoid.reshape(n_pts, n_pts, 3)

n_show = 200
show_idx = np.random.choice(gm.n, n_show, replace=False)
scale_factor = 2.0

fig_gs = go.Figure()
for idx in tqdm(show_idx, desc='Building ellipsoids'):
    ell = make_ellipsoid(gm.means[idx], gm.scales[idx] * scale_factor,
                         gm.rotations[idx], n_pts=8)
    r, g, b = (gm.colors[idx] * 255).astype(int)
    fig_gs.add_trace(go.Surface(
        x=ell[:,:,0], y=ell[:,:,2], z=ell[:,:,1],
        surfacecolor=np.ones_like(ell[:,:,0]),
        colorscale=[[0, f'rgb({r},{g},{b})'], [1, f'rgb({r},{g},{b})']],
        showscale=False, opacity=0.5,
    ))

fig_gs.update_layout(
    title=f'3D Gaussian Splatting - Ellipsoid Representation ({n_show} Gaussians)',
    scene=dict(aspectmode='data',
               xaxis_title='X', yaxis_title='Z', zaxis_title='Y'),
    width=900, height=700, showlegend=False,
)
fig_gs.show()

In [ ]:
def splat_render(gm, camera_pos, camera_target, up=np.array([0,1,0]),
                 img_size=200, fov_scale=0.18):
    """Render 3D Gaussians to 2D via splatting + alpha compositing (vectorized)."""
    forward = camera_target - camera_pos
    forward /= np.linalg.norm(forward)
    right = np.cross(forward, up)
    right /= np.linalg.norm(right)
    up_cam = np.cross(right, forward)

    # Project all means at once
    pts = gm.means - camera_pos
    px_all = pts @ right
    py_all = pts @ up_cam
    depths = pts @ forward

    # Sort back-to-front
    order = np.argsort(-depths)

    # Pre-compute 2D Jacobian
    J = np.stack([right, up_cam], axis=0)  # (2, 3)

    image = np.zeros((img_size, img_size, 4))

    for idx in order:
        cx = int((px_all[idx] / fov_scale + 0.5) * img_size)
        cy = int((0.5 - py_all[idx] / fov_scale) * img_size)

        if not (0 <= cx < img_size and 0 <= cy < img_size):
            continue

        cov2d = J @ gm.covariances[idx] @ J.T
        eigvals = np.linalg.eigvalsh(cov2d)
        radius_px = max(1, min(int(2.5 * np.sqrt(max(eigvals.max(), 1e-10)) / fov_scale * img_size), 12))

        try:
            cov2d_inv = np.linalg.inv(cov2d + np.eye(2) * 1e-8)
        except np.linalg.LinAlgError:
            continue

        alpha = gm.opacities[idx]
        color = gm.colors[idx]

        # Vectorize the inner splat loop
        dy_range = np.arange(-radius_px, radius_px + 1)
        dx_range = np.arange(-radius_px, radius_px + 1)
        ddy, ddx = np.meshgrid(dy_range, dx_range, indexing='ij')
        ny_arr = cy + ddy.ravel()
        nx_arr = cx + ddx.ravel()

        # Filter in-bounds
        mask = (nx_arr >= 0) & (nx_arr < img_size) & (ny_arr >= 0) & (ny_arr < img_size)
        nx_arr, ny_arr = nx_arr[mask], ny_arr[mask]
        ddx_f, ddy_f = ddx.ravel()[mask], ddy.ravel()[mask]

        if len(nx_arr) == 0:
            continue

        offsets = np.stack([ddx_f / img_size * fov_scale,
                           -ddy_f / img_size * fov_scale], axis=1)
        g_vals = np.exp(-0.5 * np.sum(offsets @ cov2d_inv * offsets, axis=1))
        a_vals = alpha * g_vals

        remaining = 1.0 - image[ny_arr, nx_arr, 3]
        active = remaining > 0.01
        nx_a, ny_a = nx_arr[active], ny_arr[active]
        a_a = a_vals[active]
        rem_a = remaining[active]

        image[ny_a, nx_a, :3] += (rem_a * a_a)[:, None] * color[None, :]
        image[ny_a, nx_a, 3] += rem_a * a_a

    image[:, :, :3] = np.clip(image[:, :, :3], 0, 1)
    return image

center = gm.means.mean(axis=0)
cam = center + np.array([0.0, 0.05, 0.25])
print('Rendering Gaussian splat (200x200)...')
img = splat_render(gm, cam, center, img_size=200)

fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(img[:, :, :3])
ax.set_title('2D Gaussian Splatting Render', fontsize=14)
ax.axis('off')
plt.tight_layout()
plt.show()
print(f'Pixel coverage: {(img[:,:,3] > 0.01).sum() / img[:,:,3].size * 100:.1f}%')

In [ ]:
angles = [0, 90, 180, 270]
radius = 0.25

fig, axes = plt.subplots(1, 4, figsize=(20, 5))
for i, angle_deg in enumerate(angles):
    angle = np.radians(angle_deg)
    cam_pos = center + np.array([radius * np.sin(angle), 0.05, radius * np.cos(angle)])
    print(f'  Rendering {angle_deg} deg...')
    img_view = splat_render(gm, cam_pos, center, img_size=150)
    axes[i].imshow(img_view[:, :, :3])
    axes[i].set_title(f'{angle_deg}', fontsize=14)
    axes[i].axis('off')

plt.suptitle('Multi-View Gaussian Splatting', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

---
## 5. Volume Measurement - Divergence Theorem

The **Divergence Theorem** converts volume integral to surface integral:

$$V = \iiint_\Omega dV = \frac{1}{3} \oint_S \mathbf{r} \cdot \hat{\mathbf{n}} \, dA$$

For a triangulated mesh:

$$V = \frac{1}{6} \sum_{\text{faces}} (\mathbf{v}_1 + \mathbf{v}_2 + \mathbf{v}_3) \cdot [(\mathbf{v}_2 - \mathbf{v}_1) \times (\mathbf{v}_3 - \mathbf{v}_1)]$$

Derived from choosing $\mathbf{F} = \mathbf{r}/3$ so that $\nabla \cdot \mathbf{F} = 1$.

In [ ]:
def volume_divergence_theorem(verts, faces):
    """Volume via Divergence Theorem: V = (1/6) sum (v1+v2+v3) . [(v2-v1) x (v3-v1)]"""
    v1 = verts[faces[:, 0]]
    v2 = verts[faces[:, 1]]
    v3 = verts[faces[:, 2]]
    cross = np.cross(v2 - v1, v3 - v1)
    centroid = (v1 + v2 + v3) / 3.0
    contributions = np.sum(centroid * cross, axis=1)
    volume = abs(contributions.sum()) / 6.0
    return volume, contributions

vol_div, face_contributions = volume_divergence_theorem(vertices, faces)

print('=== Divergence Theorem ===')
print(f'  Volume            = {vol_div:.10f} cubic units')
print(f'  Faces processed   : {len(faces):,}')
print(f'  Max contribution  : {abs(face_contributions).max():.10f}')
print(f'  Min contribution  : {abs(face_contributions).min():.10f}')

In [ ]:
mesh_tm = trimesh.Trimesh(vertices=vertices, faces=faces)
vol_trimesh = abs(mesh_tm.volume)

print('=== Cross Validation ===')
print(f'  Our implementation : {vol_div:.10f}')
print(f'  Trimesh reference  : {vol_trimesh:.10f}')
print(f'  Absolute diff      : {abs(vol_div - vol_trimesh):.2e}')
print(f'  Relative diff      : {abs(vol_div - vol_trimesh) / vol_trimesh * 100:.6f}%')
print()
print(f'  Watertight         : {mesh_tm.is_watertight}')
print(f'  Winding consistent : {mesh_tm.is_winding_consistent}')
print(f'  Surface area       : {mesh_tm.area:.10f}')
print(f'  Euler number       : {mesh_tm.euler_number}')

---
## 6. Additional Volume Methods

| Method | Type | Complexity |
|--------|------|------------|
| Signed Tetrahedra | Exact | O(F) |
| Monte Carlo | Stochastic | O(N) |
| Voxelization | Discrete | O(V) |
| Convex Hull | Upper bound | O(N log N) |

In [ ]:
def volume_signed_tetrahedra(verts, faces):
    """V = |sum (1/6) * v1 . (v2 x v3)|"""
    v1 = verts[faces[:, 0]]
    v2 = verts[faces[:, 1]]
    v3 = verts[faces[:, 2]]
    signed_vols = np.sum(v1 * np.cross(v2, v3), axis=1) / 6.0
    return abs(signed_vols.sum()), signed_vols

vol_tetra, tetra_contribs = volume_signed_tetrahedra(vertices, faces)
print(f'Signed Tetrahedra: {vol_tetra:.10f}')

In [ ]:
def volume_monte_carlo(verts, faces, n_samples=50_000):
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    lo, hi = verts.min(axis=0), verts.max(axis=0)
    bbox_vol = np.prod(hi - lo)
    pts = np.random.uniform(lo, hi, (n_samples, 3))
    inside = mesh.contains(pts)
    n_inside = inside.sum()
    volume = bbox_vol * (n_inside / n_samples)
    p = n_inside / n_samples
    ci_95 = 1.96 * np.sqrt(p * (1 - p) / n_samples) * bbox_vol
    return volume, n_inside, ci_95

# Use lighter mesh for ray-based containment test (much faster)
print('Running Monte Carlo (50k samples on Res2 mesh)...')
vol_mc, mc_inside, mc_ci = volume_monte_carlo(vertices_light, faces_light, 50_000)
print(f'Monte Carlo Volume  : {vol_mc:.10f}')
print(f'  95% CI            : +/- {mc_ci:.10f}')
print(f'  Inside            : {mc_inside:,} / 50,000')
print(f'  Relative error    : {abs(vol_mc - vol_div) / vol_div * 100:.4f}%')

In [ ]:
def volume_voxelization(verts, faces, pitch=0.002):
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    voxelized = mesh.voxelized(pitch=pitch)
    n_filled = voxelized.filled_count
    return n_filled * (pitch ** 3), n_filled

print('Running Voxelization (pitch=0.002)...')
vol_vox, n_vox = volume_voxelization(vertices_light, faces_light, 0.002)
print(f'Voxelization Volume : {vol_vox:.10f}')
print(f'  Filled voxels     : {n_vox:,}')
print(f'  Relative error    : {abs(vol_vox - vol_div) / vol_div * 100:.4f}%')

In [ ]:
vol_hull = hull.volume
print(f'Convex Hull Volume  : {vol_hull:.10f}')
print(f'  Compactness       : {vol_div / vol_hull * 100:.2f}% (mesh/hull)')

---
## 7. Gaussian Density Field Volume

Volume from the 3DGS representation via Monte Carlo on the density field:

$$V_{\text{GS}} \approx V_{\text{bbox}} \cdot \frac{1}{N} \sum_{i=1}^{N} \mathbb{1}[G(\mathbf{x}_i) > \tau]$$

In [ ]:
def volume_gaussian_field(gm, threshold=0.1, n_samples=30_000):
    margin = gm.scales.max() * 3
    lo = gm.means.min(axis=0) - margin
    hi = gm.means.max(axis=0) + margin
    bbox_vol = np.prod(hi - lo)
    pts = np.random.uniform(lo, hi, (n_samples, 3))

    search_radius_sq = (gm.scales.max() * 4) ** 2
    density = np.zeros(n_samples)

    for i in range(gm.n):
        diff = pts - gm.means[i]
        dists_sq = np.sum(diff ** 2, axis=1)
        close = dists_sq < search_radius_sq
        if close.any():
            cov_inv = gm.cov_invs[i]
            exponent = -0.5 * np.einsum('ij,jk,ik->i', diff[close], cov_inv, diff[close])
            density[close] += gm.opacities[i] * np.exp(exponent)

    n_inside = (density > threshold).sum()
    return bbox_vol * (n_inside / n_samples), n_inside

print('Estimating volume from Gaussian density field (30k samples)...')
vol_gs, gs_inside = volume_gaussian_field(gm, threshold=0.1, n_samples=30_000)
print(f'Gaussian Density Volume: {vol_gs:.10f}')
print(f'  Points above threshold: {gs_inside:,} / 30,000')
print(f'  vs Divergence Theorem : {abs(vol_gs - vol_div) / vol_div * 100:.2f}% diff')

---
## 8. Comparison & Final Analysis

In [ ]:
results = pd.DataFrame([
    {'Method': 'Divergence Theorem',   'Volume': vol_div,     'Type': 'Exact (surface integral)'},
    {'Method': 'Signed Tetrahedra',    'Volume': vol_tetra,   'Type': 'Exact (geometric)'},
    {'Method': 'Trimesh (reference)',   'Volume': vol_trimesh, 'Type': 'Exact (library)'},
    {'Method': 'Monte Carlo (50k)',     'Volume': vol_mc,      'Type': 'Stochastic'},
    {'Method': 'Voxelization',          'Volume': vol_vox,     'Type': 'Discrete'},
    {'Method': 'Gaussian Density',      'Volume': vol_gs,      'Type': 'Density field'},
    {'Method': 'Convex Hull (bound)',   'Volume': vol_hull,    'Type': 'Upper bound'},
])
results['Rel. Error (%)'] = abs(results['Volume'] - vol_div) / vol_div * 100

print('=' * 85)
print('  VOLUME MEASUREMENT COMPARISON  -  Stanford Bunny')
print('=' * 85)
display_df = results.copy()
display_df['Volume'] = display_df['Volume'].map(lambda x: f'{x:.10f}')
display_df['Rel. Error (%)'] = display_df['Rel. Error (%)'].map(lambda x: f'{x:.4f}')
print(display_df.to_string(index=False))
print('=' * 85)

In [ ]:
volumes = [vol_div, vol_tetra, vol_trimesh, vol_mc, vol_vox, vol_gs, vol_hull]
labels  = ['Divergence\nTheorem', 'Signed\nTetrahedra', 'Trimesh',
            'Monte\nCarlo', 'Voxel', 'Gaussian\nDensity', 'Convex\nHull']
colors  = ['#2196F3', '#1976D2', '#0D47A1', '#4CAF50', '#FF9800', '#9C27B0', '#F44336']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

ax1.bar(labels, volumes, color=colors, edgecolor='black', linewidth=0.5)
ax1.axhline(y=vol_div, color='red', linestyle='--', alpha=0.6, label='Reference')
ax1.set_ylabel('Volume (cubic units)', fontsize=12)
ax1.set_title('Volume by Method', fontsize=14, fontweight='bold')
ax1.legend()
ax1.tick_params(axis='x', labelsize=9)

errors = [abs(v - vol_div) / vol_div * 100 for v in volumes[:-1]]
ax2.bar(labels[:-1], errors, color=colors[:-1], edgecolor='black', linewidth=0.5)
ax2.set_ylabel('Relative Error (%)', fontsize=12)
ax2.set_title('Relative Error vs Divergence Theorem', fontsize=14, fontweight='bold')
ax2.tick_params(axis='x', labelsize=9)

plt.tight_layout()
plt.show()

In [ ]:
fig_contrib = go.Figure(data=[go.Mesh3d(
    x=vertices[:, 0], y=vertices[:, 2], z=vertices[:, 1],
    i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
    intensitymode='cell',
    intensity=face_contributions,
    colorscale='RdBu',
    showscale=True,
    colorbar=dict(title='Contribution'),
    flatshading=True,
)])
fig_contrib.update_layout(
    title='Divergence Theorem - Per-Face Volume Contribution',
    scene=dict(aspectmode='data'),
    width=900, height=700,
)
fig_contrib.show()

In [ ]:
print('Volume across mesh resolutions:')
print('-' * 65)
res_data = []
for label, data in meshes.items():
    v, f = data['vertices'], data['faces']
    vol, _ = volume_divergence_theorem(v, f)
    mt = trimesh.Trimesh(vertices=v, faces=f)
    res_data.append({'Resolution': label, 'Vertices': len(v), 'Faces': len(f),
                     'Volume': vol, 'Surface Area': mt.area, 'Watertight': mt.is_watertight})
    print(f'  {label:16s} | V={vol:.10f} | A={mt.area:.8f} | watertight={mt.is_watertight}')

df_res = pd.DataFrame(res_data)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
ax1.plot(df_res['Vertices'], df_res['Volume'], 'bo-', markersize=10, linewidth=2)
ax1.set_xlabel('Number of Vertices'); ax1.set_ylabel('Volume')
ax1.set_title('Volume vs Mesh Resolution', fontweight='bold'); ax1.grid(True, alpha=0.3)

ax2.plot(df_res['Vertices'], df_res['Surface Area'], 'rs-', markersize=10, linewidth=2)
ax2.set_xlabel('Number of Vertices'); ax2.set_ylabel('Surface Area')
ax2.set_title('Surface Area vs Mesh Resolution', fontweight='bold'); ax2.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
output = {
    'dataset': 'Stanford Bunny',
    'working_mesh': WORK,
    'n_vertices': int(len(vertices)),
    'n_faces': int(len(faces)),
    'volumes': {
        'divergence_theorem': float(vol_div),
        'signed_tetrahedra': float(vol_tetra),
        'trimesh_reference': float(vol_trimesh),
        'monte_carlo_50k': float(vol_mc),
        'voxelization': float(vol_vox),
        'gaussian_density': float(vol_gs),
        'convex_hull_bound': float(vol_hull),
    },
    'surface_area': float(mesh_tm.area),
    'is_watertight': bool(mesh_tm.is_watertight),
    'gaussian_model': {'n_gaussians': int(gm.n), 'sample_rate': 0.5},
}

with open('bunny_volumetric_results.json', 'w') as f:
    json.dump(output, f, indent=2)

print('Results saved to bunny_volumetric_results.json')
print()
print('=' * 60)
print('  ANALYSIS COMPLETE')
print('=' * 60)
print(f'  Best volume estimate : {vol_div:.10f}')
print(f'  Methods compared     : 7')
print(f'  Mesh faces           : {len(faces):,}')
print(f'  3DGS Gaussians       : {gm.n:,}')
print('=' * 60)